# Check if the packet loss data is abnormal

This notebook finds out all the links which have at least five packet loss measurements in the past one hour and the average value of the packet loss measurements is greater than 2%. It is run by a cron job every hour, and it will write the detailed information of every alarm into Elastic Search with the _index: alarms-year-month and _type: packetloss.

### import all the packages needed for this task

In [1]:
rom elasticsearch import Elasticsearch, exceptions as es_exceptions, helpers
import sys
import datetimef

### If needed to calculate packet loss for other time moment than "now", overwrite it bellow

In [2]:
cdt = datetime.datetime.utcnow()
#cdt = datetime.datetime(2017,1,21,9,0,0)

GT = (cdt - datetime.timedelta(hours=3)).strftime("%Y%m%dT%H%m%S+0000")
LT = cdt.strftime("%Y%m%dT%H%m%S+0000")
print('between: ', GT, ' and ', LT)

between:  20170121T060100+0000  and  20170121T090100+0000


### establish the Elastic Search connection

In [3]:
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

### define functions to write an alarm record into ES with detailed info

In [4]:
ipSite={}  # for mapping IP to Site name
toAlertOn=[]

def generate_doc(src_site_ip, dest_site_ip, measurements, avgpl):
   if src_site_ip not in ipSite:
        print('serious source mapping issue')
        return
   if dest_site_ip not in ipSite:
        print('serious destination mapping issue')
        return
   
   doc = {
        '_index':get_index_name(),
        '_type' : 'packetloss',
        'src' : src_site_ip,
        'dest' : dest_site_ip,
        'srcSite' : ipSite[src_site_ip],
        'destSite' : ipSite[dest_site_ip],
        'alarmTime' : int( (cdt-datetime.datetime(1970,1,1) ).total_seconds() * 1000 ),
        'measurements' : measurements,
        'packetLossAvg' : avgpl
   }
   return doc

def get_index_name():
    date = cdt.strftime("%Y-%m")   # date format is yyyy-mm
    index_name = 'alarms-'+date
    return index_name



### list all network\_weather* indices

In [5]:
indices = es.cat.indices(index="network_weather-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

### find indices to be used

In [6]:
# searching through last three indices as this simplifies utc issues
pday  = cdt - datetime.timedelta(days=1)
nday  = cdt + datetime.timedelta(days=1)
ind1 = 'network_weather-%02d.%d.%d' % (cdt.year, cdt.month, cdt.day)
ind2 = 'network_weather-%d.%d.%d' % (pday.year, pday.month, pday.day)
ind3 = 'network_weather-%d.%d.%d' % (nday.year, nday.month, nday.day)

ind=[]
if ind1 in indices :
   ind.append(ind1)

if ind2 in indices:
   ind.append(ind2)

if ind3 in indices:
   ind.append(ind3)

if len(ind)==0:
   print ('no current indices found. Aborting.')
   sys.exit(1)
else:
   print('will use indices:', ind)

will use indices: ['network_weather-2017.1.21', 'network_weather-2017.1.20']


### get aggregated data for the past 2 hours
This query is composed of 3 parts: a) filter - takes only packet loss data, and production servers in last 1h. b) aggregation -  finds average packet loss per source and destination c) finds IP to site name mapping (both source and destination) 

In [7]:
query={
   "size": 0,
   "query": {
    "bool": {
      "must": [
        {"term": { "_type" : "packet_loss_rate"}},
        {"term": { "srcProduction" : True }},
        {"term": { "destProduction" : True }}
      ],
      "filter" : {
        "range" : {
          "timestamp" :{ "gt" : GT, "lt" : LT }
        }
      }
    }
   },
    "aggs" : {
      "src" : {
        "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "dest" : {
            "terms" : {"field" : "dest", "size": 1000},
            "aggs" : {
              "avgpl" : {
                "avg" :{
                  "field" : "packet_loss"
              }
            }
          }
        }
      }
    },
    "srcSites" : {
      "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "srcsitename" : {
            "terms" : { "field" : "srcSite" }
        }
      }
    },
    "destSites" : {
      "terms" : { "field" : "dest", "size": 1000 },
        "aggs" : {
          "destsitename" : {
            "terms" : { "field" : "destSite" }
        }
      }
    }
  }
}

#execute query
res = es.search(index=ind, body=query, request_timeout=120)
#print(res)

### proces IP to site name mapping data

In [8]:
srcsites=res['aggregations']['srcSites']['buckets']
#print(srcsites)
for sS in srcsites:
   #print(sS)
   siteName=sS['srcsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[sS['key']]=siteName

destsites=res['aggregations']['destSites']['buckets']
#print(destsites)
for dS in destsites:
   #print(dS)
   siteName=dS['destsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[dS['key']]=siteName

print(ipSite)


{'81.180.86.38': 'RO-02-NIPNE', '144.16.111.26': 'INDIACMS-TIFR', '192.101.107.152': 'UnknownSite', '130.246.176.109': 'RAL-LCG2', '193.48.99.76': 'IN2P3-CC', '131.225.2.47': 'UnknownSite', '152.84.101.141': 'UnknownSite', '194.85.69.75': 'ITEP', '64.57.30.143': 'UnknownSite', '134.158.150.245': 'IN2P3-IRES', '193.144.80.12': 'UnknownSite', '129.93.5.165': 'Nebraska', '192.170.227.160': 'MWT2', '146.83.90.7': 'EELA-UTFSM', '72.36.96.15': 'MWT2', '149.165.225.223': 'MWT2', '192.41.236.31': 'AGLT2', '193.60.193.3': 'UKI-SCOTGRID-DURHAM', '192.101.161.186': 'UnknownSite', '193.48.83.165': 'IN2P3-LPSC', '206.12.24.251': 'SFU-LCG2', '194.80.35.169': 'UKI-NORTHGRID-LANCS-HEP', '206.12.9.2': 'TRIUMF-LCG2', '193.109.172.188': 'pic', '143.215.129.69': 'UnknownSite', '198.124.80.201': 'UnknownSite', '132.230.202.235': 'UNI-FREIBURG', '144.92.180.75': 'GLOW', '192.108.47.12': 'FZK-LCG2', '194.190.165.192': 'RU-Protvino-IHEP', '148.6.8.251': 'BUDAPEST', '138.253.60.82': 'UKI-NORTHGRID-LIV-HEP', '1

### process packet loss averages

In [9]:
src=res['aggregations']['src']['buckets']
#print(src)

for s in src:
   #print(s)
   source=s['key']
   for d in s['dest']['buckets']:
      destination=d['key']
      avgpl=d['avgpl']['value']
      docs=d['doc_count']
#      print(source, destination, docs, avgpl)
      if avgpl > 0.02 and docs > 4:
         toAlertOn.append(generate_doc(source, destination, docs, avgpl))

for alert in toAlertOn:
   print(alert)

{'src': '142.150.19.61', 'dest': '81.180.86.64', '_type': 'packetloss', 'destSite': 'RO-07-NIPNE', 'measurements': 86, 'srcSite': 'CA-SCINET-T2', 'packetLossAvg': 0.04788404398836022, 'alarmTime': 1484989200000, '_index': 'alarms-2017-01'}
{'src': '142.150.19.61', 'dest': '145.100.17.8', '_type': 'packetloss', 'destSite': 'SARA-MATRIX', 'measurements': 43, 'srcSite': 'CA-SCINET-T2', 'packetLossAvg': 0.0408430234204198, 'alarmTime': 1484989200000, '_index': 'alarms-2017-01'}
{'src': '142.150.19.61', 'dest': '131.169.98.30', '_type': 'packetloss', 'destSite': 'DESY-HH', 'measurements': 36, 'srcSite': 'CA-SCINET-T2', 'packetLossAvg': 0.04562114184308383, 'alarmTime': 1484989200000, '_index': 'alarms-2017-01'}
{'src': '142.150.19.61', 'dest': '81.180.86.38', '_type': 'packetloss', 'destSite': 'RO-02-NIPNE', 'measurements': 36, 'srcSite': 'CA-SCINET-T2', 'packetLossAvg': 0.04639814885902322, 'alarmTime': 1484989200000, '_index': 'alarms-2017-01'}
{'src': '206.12.154.60', 'dest': '141.108.35

### write alarms to Elasticsearch

In [10]:
try:
   res = helpers.bulk(es, toAlertOn, raise_on_exception=True,request_timeout=60)
   print("inserted:",res[0], '\tErrors:',res[1])
except es_exceptions.ConnectionError as e:
   print('ConnectionError ', e)
except es_exceptions.TransportError as e:
   print('TransportError ', e)
except helpers.BulkIndexError as e:
   print(e[0])
   for i in e[1]:
      print(i)
except:
   print('Something seriously wrong happened.')

inserted: 35 	Errors: []
